Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [2]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [38]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        
    def nim_reset(self) -> None:
        self._rows = [i * 2 + 1 for i in range(len(self._rows))]
        self._k = None


## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [6]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [7]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [8]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=4)
INFO:root:status: <1 3 5 7 5>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=5)
INFO:root:status: <1 3 5 2 5>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 2 5 2 5>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 2 5 2 5>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=4)
INFO:root:status: <0 2 5 2 1>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 2 5 1 1>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 2 5 0 1>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <0 2 3 0 1>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <0 0 3 0 1>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=3)
INFO:root:status: <0 0 0 0 1>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 0

### Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)

In [49]:
def NimSum_Agent(state: Nim) -> Nimply:
    current_nim_sum = nim_sum(state)
    
    if current_nim_sum == 0:
        # First turn : Pick it randomly
        return pure_random(state)  

    # Second turn : Pick the move that makes the nim sum 0 (not first turn)
    for row in range(len(state.rows)):
        for num_objects in range(1, state.rows[row] + 1):
            temp_state = deepcopy(state)
            temp_state.nimming(Nimply(row, num_objects))
            if nim_sum(temp_state) != 0:
                return Nimply(row, num_objects)

    # Worst case scenario : Pick it randomly
    return pure_random(state)


In [51]:
logging.getLogger().setLevel(logging.INFO)

strategies = [(NimSum_Agent, pure_random), (NimSum_Agent, gabriele), (NimSum_Agent, optimal)]

nims = [Nim(k) for k in [3,4,5]]

# LET'S PLAY A LOT !

for nim in nims:
    for strategy in strategies:
        nim.nim_reset()
        logging.info(f"init : {nim}")
        player = 0
        while nim:
            ply = strategy[player](nim)
            logging.info(f"ply: player {player} plays {ply}")
            nim.nimming(ply)
            logging.info(f"status: {nim}")
            player = 1 - player
        logging.info(f"status: Player {player} won!")
        logging.info(f"{strategy[player].__name__} won against {strategy[1-player].__name__} with {nim.rows[0]} rows")
        logging.info(f"\n")


INFO:root:init : <1 3 5>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=5)
INFO:root:status: <0 3 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=2)
INFO:root:status: <0 0 0>
INFO:root:status: Player 0 won!
INFO:root:NimSum_Agent won against pure_random with 0 rows
INFO:root:

INFO:root:init : <1 3 5>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=3)
INFO:root:status: <0 0 5>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0 4>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=4)
INFO:root:status: <0 0 0>
INFO:root:status: Player 0 won!
INFO:root:NimSum_Agent won against gabriele with 0 rows
INFO:root:

INFO:root:init : <1 3 5>
INFO:root:ply: player 0 plays Nimply(row=0

### Task2.2: An agent using evolved rules using ES

In [3]:
from typing import List, Tuple, Optional, Callable

Nimply = namedtuple("Nimply", "row, num_objects")

class Agent:
    """
    Represents an agent/player in the Nim game with a strategy based on genetic parameters.
    """
    def __init__(self, num_rows: int, params: Optional[np.ndarray] = None, k: Optional[int] = None):
        """
        Initializes an Agent for the Nim game.

        :param num_rows: Number of rows in the Nim game.
        :param params: Array of parameters defining the agent's strategy.
        :param k: Maximum number of objects that can be taken in one move.
        """
        self.num_rows = num_rows
        self.k = k or (2 * num_rows - 1)
        self._params = params or np.random.normal(size=num_rows)
        self.fitness_scores = []

    @property
    def fitness(self) -> float:
        """
        Calculates the average fitness score of the agent.

        :return: Average fitness score.
        """
        return np.mean(self.fitness_scores) if self.fitness_scores else 0

    def strategy(self, nim: 'Nim') -> Nimply:
        """
        Defines the strategy of the agent to play the Nim game.

        :param nim: Instance of the Nim game.
        :return: Chosen move (Nimply) based on the agent's strategy.
        """
        # Example strategy implementation goes here
        # This should be replaced with the actual strategy logic
        return Nimply(random.choice(range(self.num_rows)), random.randint(1, self.k))

    def update_fitness(self, fitness: float) -> None:
        """
        Updates the fitness score of the agent.

        :param fitness: The new fitness score to be added.
        """
        self.fitness_scores.append(fitness)

    def mutate(self, scale: float = 1.0, loc: float = 0.0) -> None:
        """
        Mutates the agent's parameters using Gaussian mutation.

        :param scale: Scale of the Gaussian distribution.
        :param loc: Mean of the Gaussian distribution.
        """
        self._params += np.random.normal(loc=loc, scale=scale, size=self._params.shape)


class EvolutionTask:
    """
    Manages the evolutionary process for the Nim game agents.
    """
    def __init__(self, num_rows: int, population_size: int = 50, mu: int = 25, k: Optional[int] = None):
        """
        Initializes the EvolutionTask for Nim agents.

        :param num_rows: Number of rows in the Nim game.
        :param population_size: Number of agents in the population.
        :param mu: Number of parents selected for the next generation.
        :param k: Maximum number of objects that can be taken in one move.
        """
        self.num_rows = num_rows
        self.population_size = population_size
        self.mu = mu
        self.k = k or (2 * num_rows - 1)
        self.agents = [Agent(num_rows, k=self.k) for _ in range(population_size)]

    def simulate_game(self, agent1: Agent, agent2: Agent) -> Tuple[float, float]:
        """
        Simulates a game between two agents and calculates their fitness.

        :param agent1: First agent.
        :param agent2: Second agent.
        :return: Tuple of fitness scores for agent1 and agent2.
        """
        # Game simulation and fitness calculation logic goes here
        # This should be replaced with the actual game simulation logic
        return random.random(), random.random()

    def run_generation(self) -> None:
        """
        Runs a single generation in the evolutionary process.
        """
        # Selection, crossover, mutation, and fitness evaluation logic goes here
        # Implement the evolutionary steps for one generation

    def train(self, n_generations: int = 100) -> None:
        """
        Trains the agents over multiple generations.

        :param n_generations: Number of generations to run the training for.
        """
        for _ in range(n_generations):
            self.run_generation()
            # Additional logging or tracking can be added here

# Additional code for the Nim game, utility functions, and the rest of the implementation goes here


In [7]:
et = EvolutionTask(3, mu=25, population_size=50)
et.train(n_generations=100)

In [12]:
print(et.agents)

[<__main__.Agent object at 0x0000022C10EE6E90>, <__main__.Agent object at 0x0000022C10EE6B10>, <__main__.Agent object at 0x0000022C10EE6250>, <__main__.Agent object at 0x0000022C10EE7D50>, <__main__.Agent object at 0x0000022C10EE7590>, <__main__.Agent object at 0x0000022C10EE7750>, <__main__.Agent object at 0x0000022C10EE7710>, <__main__.Agent object at 0x0000022C10EE7850>, <__main__.Agent object at 0x0000022C10EE7A90>, <__main__.Agent object at 0x0000022C10EE7510>, <__main__.Agent object at 0x0000022C10EE7B10>, <__main__.Agent object at 0x0000022C10EE5710>, <__main__.Agent object at 0x0000022C10EE7AD0>, <__main__.Agent object at 0x0000022C10EE58D0>, <__main__.Agent object at 0x0000022C10EE7E50>, <__main__.Agent object at 0x0000022C10EE4450>, <__main__.Agent object at 0x0000022C10EE7090>, <__main__.Agent object at 0x0000022C10EE5150>, <__main__.Agent object at 0x0000022C10EE52D0>, <__main__.Agent object at 0x0000022C10EE4DD0>, <__main__.Agent object at 0x0000022C10EE7290>, <__main__.Ag